# Features

In [ ]:
from datetime import datetime, date, time, timedelta

<h6>Momentos del dia</h6>

Devuelve el ratio de los momentos del dia en la ultima semana.
Ademas, devuelve: - La ultima fecha registrada, la ultima fecha registrada - 7 dias, la cantidad total de visitas en esa semana y la cantidad total de visitas ever.

In [ ]:
def feature_RatioDayMoments(df):
    #Me quedo con fechas maximas y minimas de cada user
    temp = df_pageviews
    temp2 = df_pageviews.groupby(['USER_ID']).agg({"FEC_EVENT": [max, "count"]})
    temp2.columns = ['FEC_EVENT','CANT_TOTAL_EVER']
    temp2.reset_index(inplace=True) #Reseteo el indice por lo anterior

    #Left join con mi df temporal
    temp = pd.merge(temp, temp2, how='left', left_on=['USER_ID'], right_on=['USER_ID'], suffixes=('', '_MAX')) #Agrego fecha max
    temp['FEC_EVENT_MENOS_7D'] = temp.FEC_EVENT_MAX - timedelta(days=7) #Agrego fecha min, que seria 7 dias menos
    temp = temp[(temp.FEC_EVENT >= temp.FEC_EVENT_MENOS_7D) & (temp.FEC_EVENT <= temp.FEC_EVENT_MAX)] #filtro y me quedo con la info que me interesa

    # Me calculo la cant total de visitas de los ultimos 7 dias
    temp3 = temp.groupby(['USER_ID'])['FEC_EVENT'].count() # Traigo la cant total de la semana
    temp3 = temp3.to_frame() #lo paso a dataframe (proque la anterior devuelve una serie)
    temp3.columns = ['CANT_TOTAL_ULT_7D']
    temp3.reset_index(inplace=True)
    temp = pd.merge(temp, temp3, how='left', left_on=['USER_ID'], right_on=['USER_ID'])

    #Corto el dia en 4 partes    
    temp.loc[(temp['FEC_EVENT'].dt.hour >= 18), 'MOMENTO_DIA'] = 4
    temp.loc[(temp['FEC_EVENT'].dt.hour >= 12) & (temp['FEC_EVENT'].dt.hour <18), 'MOMENTO_DIA'] = 3
    temp.loc[(temp['FEC_EVENT'].dt.hour >= 6) & (temp['FEC_EVENT'].dt.hour <12), 'MOMENTO_DIA'] = 2
    temp.loc[(temp['FEC_EVENT'].dt.hour >= 0) & (temp['FEC_EVENT'].dt.hour <6), 'MOMENTO_DIA'] = 1

    #Calculo los ratios en el momento del dia
    temp4 = pd.crosstab(temp.USER_ID, temp.MOMENTO_DIA) # * Realizamos una tabla cruzada de la Variable por Usuario colocando la frecuencia de cada valor posible como columna 
    temp4.columns = ['RATIO_VISITA_MADRUGADA','RATIO_VISITA_MANANA','RATIO_VISITA_TARDE','RATIO_VISITA_NOCHE'] 
    temp4 = temp4.apply(lambda x: x / x.sum(), axis=1)
    temp4.reset_index(inplace=True)
    temp = pd.merge(temp, temp4, how='left', left_on=['USER_ID'], right_on=['USER_ID'])

    #Preparo las columnas que voy a agregar al df y se las agrego
    temp = temp[['USER_ID','FEC_EVENT_MAX','CANT_TOTAL_EVER','CANT_TOTAL_ULT_7D','FEC_EVENT_MENOS_7D','RATIO_VISITA_MADRUGADA','RATIO_VISITA_MANANA','RATIO_VISITA_TARDE','RATIO_VISITA_NOCHE']]
    temp = temp.drop_duplicates()
    df = pd.merge(df, temp, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df = df.drop(['USER_ID'], axis = 1)
    
    return df